In [1]:
# importing pytorch and related package
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import matplotlib.pyplot as plt
import numpy as np

In [105]:
import os
from os import walk
# return a tuple of arrays ([all directories inside input path], [all files inside input path])
def get_content_of_path(pathname, get_dir=True, get_filenames=True):
    f = []
    for (dirpath, dirnames, filenames) in walk(pathname):
        f.extend(filenames)
        break
    result = []
    if get_dir is False:
        return filenames
    if get_filenames is False:
        return dirnames
    return dirnames, filenames

def get_image_lists_from_dataset(datasets_path):
    train_bmode_list = []
    train_mask_list = []
    test_bmode_list = []
    test_mask_list = []

    if datasets_path != None:
        dataset_list = os.listdir(datasets_path)
        for dataset in dataset_list:
            bmode_list = [] # input
            mask_list = [] # output
            dataset_folder = datasets_path + dataset + "/"
            dataset_dirnames = get_content_of_path(dataset_folder, get_filenames=False)

            # loop through image files and put them in list corresponding to their filename
            for filename in dataset_dirnames:
                if filename != "bmode" and filename != "mask":
                    continue
                image_folder = dataset_folder + filename + "/"
                image_paths = get_content_of_path(image_folder, get_dir=False)
                image_paths = [dataset_folder + image_path for image_path in image_paths]
                if filename == "bmode":
                    bmode_list = image_paths
                elif filename == "mask":
                    mask_list = image_paths

            # assign bmode and mask list depending on dataset type
            if dataset == "train":
                train_bmode_list = bmode_list
                train_mask_list = mask_list
            elif dataset == "test":
                test_bmode_list = bmode_list
                test_mask_list = mask_list
    return train_bmode_list, train_mask_list, test_bmode_list, test_mask_list

In [106]:
datasets_path = "/gpfs/fs2/scratch/mdoyley_lab/GPU_VA/DL_dataset/"
get_image_lists_from_dataset(datasets_path)


(['/gpfs/fs2/scratch/mdoyley_lab/GPU_VA/DL_dataset/train/VNS041_CCATL_231.png',
  '/gpfs/fs2/scratch/mdoyley_lab/GPU_VA/DL_dataset/train/VNS045_CCATL_126.png',
  '/gpfs/fs2/scratch/mdoyley_lab/GPU_VA/DL_dataset/train/VNS065_CCATR_36.png',
  '/gpfs/fs2/scratch/mdoyley_lab/GPU_VA/DL_dataset/train/VNS041_CCATL_141.png',
  '/gpfs/fs2/scratch/mdoyley_lab/GPU_VA/DL_dataset/train/VNS065_CCATR_151.png',
  '/gpfs/fs2/scratch/mdoyley_lab/GPU_VA/DL_dataset/train/VNS041_CCATL_246.png',
  '/gpfs/fs2/scratch/mdoyley_lab/GPU_VA/DL_dataset/train/VNS065_CCATR_206.png',
  '/gpfs/fs2/scratch/mdoyley_lab/GPU_VA/DL_dataset/train/VNS045_CCATL_116.png',
  '/gpfs/fs2/scratch/mdoyley_lab/GPU_VA/DL_dataset/train/VNS065_CCATL_106.png',
  '/gpfs/fs2/scratch/mdoyley_lab/GPU_VA/DL_dataset/train/VNS045_CCATR_156.png',
  '/gpfs/fs2/scratch/mdoyley_lab/GPU_VA/DL_dataset/train/VNS065_CCATR_11.png',
  '/gpfs/fs2/scratch/mdoyley_lab/GPU_VA/DL_dataset/train/VNS041_CCATR_131.png',
  '/gpfs/fs2/scratch/mdoyley_lab/GPU_VA/DL

In [9]:
class deconvol(nn.Module):
    def __init__(self, out_channels, kernel_size ,num_layer,name,channel_change, bias=True):
        super().__init__()
        self.sequence = nn.sequential()
        sequence.add_module('Max Unpooling'+name,nn.MaxUnpool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False))
        for i in range(num_layer-1):
            sequence.add_module('Deconv'+name+str(i),nn.ConvTranspose2d(out_channels, out_channels, kernel_size=kernel_size, bias=bias))
        if (channel_change):
            sequence.add_module('Deconv'+name+str(i),nn.ConvTranspose2d(out_channels, out_channels/2, kernel_size=kernel_size, bias=bias))
        else:
            sequence.add_module('Deconv'+name+str(i),nn.ConvTranspose2d(out_channels, out_channels, kernel_size=kernel_size, bias=bias))
    def forward(self,x):
        return self.sequence(x)

In [10]:
import torchvision.models as models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ft = models.vgg16(pretrained=True)

In [11]:
print(device)

cpu


In [12]:
model_encode = nn.Sequential(*[model_ft.features])
print(model_encode)

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [13]:
for param in model_encode.parameters():
    param.requires_grad = False

In [14]:
class main_model(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size ,num_layer,name, bias=True):
        super().__init__()
        self.sequence = model_encode
        sequence.add_module(nn.Flatten())
        sequence.add_module(nn.Linear(in_features=25088, out_features=4096, bias=True))
        sequence.add_module(nn.Linear(in_features=4096, out_features=4096, bias=True))
        sequence.add_module(nn.Linear(in_features=4096, out_features=25088, bias=True))
        self.sequence1 = nn.Sequential()
        sequence1.add_module(deconvol(512, 3 ,3,'decov1',False, bias=True))
        sequence1.add_module(deconvol(512, 3 ,3,'decov2',True, bias=True))
        sequence1.add_module(deconvol(256, 3 ,3,'decov3',False, bias=True))
        sequence1.add_module(deconvol(128, 3 ,2,'decov4',False, bias=True))
        sequence1.add_module(deconvol(64, 3 ,2,'decov5',True, bias=True))
        sequence1.add_module(nn.ConvTranspose2d(64, 64, kernel_size=1,stride=1, padding=1, bias=bias))
    def forward(self,x):
        x=self.sequence(x)
        x=torch.reshape(x, (None, 7, 7, 512))
        return self.sequence1(x)

In [15]:
def get_model():
    model = main_model()
    return model, optim.SGD(model.parameters(), lr=lr)

In [16]:
def loss_batch(model, xb, yb, loss_func=F.cross_entropy, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

In [17]:
def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        model.train()
        loss1 = []
        num = []
        for xb, yb in train_dl:
            losses, nums = loss_batch(model, loss_func, xb, yb)
        loss1.append(losses)
        num.append(nums)
        train_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

        print(epoch, val_loss, train_loss)

In [18]:
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
def get_data(x_train, y_train, x_valid, y_valid, bs):
    train_ds = TensorDataset(x_train, y_train)
    train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)
    valid_ds = TensorDataset(x_valid, y_valid)
    valid_dl = DataLoader(valid_ds, batch_size=bs * 2)
    return (train_dl,valid_dl)